# Minneapolis, MN Simulations
8760 rows per simulation<br>
196 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 0.83
elec = 0.1314

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 1716960  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#1716960  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

all_data2

#1716960  rows, 6 columns

,Scenario,Date_Time,Illuminance(lux),FanEnergy[J](Hourly),HeatingEnergy[J](Hourly),CoolingEnergy[J](Hourly)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,431563.2470,27855626.71,0.0
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,482963.2708,31173286.14,0.0
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,494677.8003,31929410.68,0.0
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,493311.7474,31841237.61,0.0
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,502462.3607,32431871.93,0.0
...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,290533.6283,18752746.80,0.0
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,289338.8894,18675631.33,0.0
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,299605.3203,19338287.08,0.0
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,313127.1056,20211062.52,0.0


-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_annual

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.409043,26.402040,0.0
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.457761,29.546574,0.0
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.468864,30.263242,0.0
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.467569,30.179670,0.0
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.476242,30.739483,0.0
...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.275373,17.774174,0.0
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.274240,17.701083,0.0
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.283971,18.329160,0.0
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.296787,19.156391,0.0


---

## Get Annual Cost Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [12]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [14]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_annual_costs

,Scenario,Date_Time,FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.015752,0.219137,0.0
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.017628,0.245237,0.0
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.018056,0.251185,0.0
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.018006,0.250491,0.0
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.018340,0.255138,0.0
...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.010604,0.147526,0.0
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.010561,0.146919,0.0
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.010936,0.152132,0.0
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.011429,0.158998,0.0


---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Add baseline data <br>
Export combine file

In [15]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)
allAnnualData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.409043,26.402040,0.0,0.015752,0.219137,0.0
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.457761,29.546574,0.0,0.017628,0.245237,0.0
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.468864,30.263242,0.0,0.018056,0.251185,0.0
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.467569,30.179670,0.0,0.018006,0.250491,0.0
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.476242,30.739483,0.0,0.018340,0.255138,0.0
...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.275373,17.774174,0.0,0.010604,0.147526,0.0
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.274240,17.701083,0.0,0.010561,0.146919,0.0
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.283971,18.329160,0.0,0.010936,0.152132,0.0
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.296787,19.156391,0.0,0.011429,0.158998,0.0


In [16]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

allAnnualData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.409043,26.402040,0.0,0.015752,0.219137,0.0,26.811083,0.234889
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.457761,29.546574,0.0,0.017628,0.245237,0.0,30.004335,0.262865
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.468864,30.263242,0.0,0.018056,0.251185,0.0,30.732106,0.269241
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.467569,30.179670,0.0,0.018006,0.250491,0.0,30.647239,0.268497
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.476242,30.739483,0.0,0.018340,0.255138,0.0,31.215726,0.273478
...,...,...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.275373,17.774174,0.0,0.010604,0.147526,0.0,18.049547,0.158130
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.274240,17.701083,0.0,0.010561,0.146919,0.0,17.975323,0.157480
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.283971,18.329160,0.0,0.010936,0.152132,0.0,18.613131,0.163068
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.296787,19.156391,0.0,0.011429,0.158998,0.0,19.453178,0.170427



### Establish Annual Energy and Cost Baseline Data


In [17]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_annual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baseline_annual

In [18]:
annualEnergy_fan = baseline_annual["FanEnergy(kBtu)"].sum()
annualEnergy_fan

2518.5942054690404

In [19]:
annualEnergy_cool = baseline_annual["CoolingEnergy(kBtu)"].sum()
annualEnergy_cool

9148.732496988963

In [20]:
annualEnergy_heat = baseline_annual["HeatingEnergy(kBtu)"].sum()
annualEnergy_heat

106490.68918688194

In [21]:
annualEnergy = (annualEnergy_fan + annualEnergy_cool + annualEnergy_heat)
annualEnergy

118158.01588933993

In [22]:
annualCost_fan = baseline_annual["FanEnergy($)"].sum()
annualCost_fan

96.98990093485853

In [23]:
annualCost_cool = baseline_annual["CoolingEnergy($)"].sum()
annualCost_cool

352.31346782092317

In [24]:
annualCost_heat = baseline_annual["HeatingEnergy($)"].sum()
annualCost_heat

883.87272025112

In [25]:
annualCost = (annualCost_fan + annualCost_cool + annualCost_heat)
annualCost

1333.1760890069017

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Add baselines to dataframe<br>
Export to file

In [26]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"], as_index=True)

#annualData.sum().round(2)

In [27]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [28]:
# Add columns for delta from base
annualDataFinal["DeltaEnergyFromBase"] = annualDataFinal["AnnualEnergy(kBtu)"] - annualEnergy
annualDataFinal["DeltaCostFromBase"] = annualDataFinal["AnnualCost($)"] - annualCost

annualDataFinal.round(2)

,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($),DeltaEnergyFromBase,DeltaCostFromBase
Scenario,,,,,,,,,,,
Clear/Clear,19590610.39,2707.08,108952.04,10670.85,104.25,904.30,410.93,122329.97,1419.48,4171.95,86.30
Clear/Clear-BBG-15,9240018.13,2596.47,118584.93,8048.82,99.99,984.25,309.96,129230.22,1394.20,11072.20,61.02
Clear/Clear-BBG-30,13336348.07,2567.51,115619.31,8217.09,98.87,959.64,316.44,126403.91,1374.95,8245.89,41.77
Clear/Clear-BBG-45,14035322.41,2525.38,114521.89,7959.01,97.25,950.53,306.50,125006.28,1354.28,6848.26,21.10
Clear/Clear-BBG-60,8724318.09,2457.29,115052.26,7181.01,94.63,954.93,276.54,124690.56,1326.10,6532.54,-7.08
...,...,...,...,...,...,...,...,...,...,...,...
lowE(2)/lowE(4)-argon-VB15,3344783.82,2143.17,101132.39,6145.11,82.53,839.40,236.65,109420.68,1158.58,-8737.34,-174.60
lowE(2)/lowE(4)-argon-VB30,1952148.69,2135.31,101979.61,5929.20,82.23,846.43,228.33,110044.11,1156.99,-8113.91,-176.19
lowE(2)/lowE(4)-argon-VB45,1173662.55,2125.16,102903.17,5676.67,81.84,854.10,218.61,110705.00,1154.54,-7453.02,-178.64


In [29]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/msp_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [30]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [31]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [32]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.119879,0.264020,0.0
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.134156,0.295466,0.0
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.137411,0.302632,0.0
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.137031,0.301797,0.0
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.139573,0.307395,0.0
...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.080704,0.177742,0.0
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.080372,0.177011,0.0
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.083224,0.183292,0.0
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.086980,0.191564,0.0


## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [33]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [34]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [35]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Establish baseline <br>
Export combine file

In [36]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.119879,0.264020,0.0,0.015752,0.219137,0.0
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.134156,0.295466,0.0,0.017628,0.245237,0.0
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.137411,0.302632,0.0,0.018056,0.251185,0.0
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.137031,0.301797,0.0,0.018006,0.250491,0.0
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.139573,0.307395,0.0,0.018340,0.255138,0.0
...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.080704,0.177742,0.0,0.010604,0.147526,0.0
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.080372,0.177011,0.0,0.010561,0.146919,0.0
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.083224,0.183292,0.0,0.010936,0.152132,0.0
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.086980,0.191564,0.0,0.011429,0.158998,0.0


In [37]:
# Add cost totals, don't total energy...in different units
totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

#allMonthlyData

In [38]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyCost($),Month
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.119879,0.264020,0.0,0.015752,0.219137,0.0,0.234889,01
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.134156,0.295466,0.0,0.017628,0.245237,0.0,0.262865,01
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.137411,0.302632,0.0,0.018056,0.251185,0.0,0.269241,01
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.137031,0.301797,0.0,0.018006,0.250491,0.0,0.268497,01
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.139573,0.307395,0.0,0.018340,0.255138,0.0,0.273478,01
...,...,...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.080704,0.177742,0.0,0.010604,0.147526,0.0,0.158130,12
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.080372,0.177011,0.0,0.010561,0.146919,0.0,0.157480,12
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.083224,0.183292,0.0,0.010936,0.152132,0.0,0.163068,12
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.086980,0.191564,0.0,0.011429,0.158998,0.0,0.170427,12



### Establish Monthly Energy and Cost Baseline Data


In [39]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_monthly = allMonthlyData[allMonthlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_monthly

In [40]:
monthlyEnergy_fan = baseline_monthly["FanEnergy(kWh)"].sum()
monthlyEnergy_fan

738.1270999608715

In [41]:
monthlyEnergy_cool = baseline_monthly["CoolingEnergy(kWh)"].sum()
monthlyEnergy_cool

2681.228826643251

In [42]:
monthlyEnergy_heat = baseline_monthly["HeatingEnergy(therms)"].sum()
monthlyEnergy_heat

1064.9068918688192

In [43]:
monthlyCost_fan = baseline_monthly["FanEnergy($)"].sum()
monthlyCost_fan

96.98990093485853

In [44]:
monthlyCost_cool = baseline_monthly["CoolingEnergy($)"].sum()
monthlyCost_cool

352.31346782092317

In [45]:
monthlyCost_heat = baseline_monthly["HeatingEnergy($)"].sum()
monthlyCost_heat

883.87272025112

In [46]:
monthlyCost = (monthlyCost_fan + monthlyCost_cool + monthlyCost_heat)
monthlyCost

1333.1760890069017

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Add baselines to dataframe<br>
Export to file

In [47]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData.sum()

In [48]:
MonthlyDataFinal = MonthlyData2.sum()

#MonthlyDataFinal

In [49]:
# Add columns for delta from base
MonthlyDataFinal["DeltaFanFromBase"] = MonthlyDataFinal["FanEnergy(kWh)"] - monthlyEnergy_fan
MonthlyDataFinal["DeltaCoolFromBase"] = MonthlyDataFinal["CoolingEnergy(kWh)"] - monthlyEnergy_cool
MonthlyDataFinal["DeltaHeatFromBase"] = MonthlyDataFinal["HeatingEnergy(therms)"] - monthlyEnergy_heat
MonthlyDataFinal["DeltaCostFromBase"] = MonthlyDataFinal["MonthlyCost($)"] - monthlyCost

MonthlyDataFinal.round(2)

Illuminance(lux)  FanEnergy(kWh)  \
Scenario                   Month                                     
Clear/Clear                 01          1769531.66          126.55   
                            02          1490278.51           78.17   
                            03          2996082.08           54.10   
                            04          1467056.60           40.79   
                            05           828620.36           38.29   
...                                            ...             ...   
lowE(2)/lowE(4)-argon-VB75  08            26509.22           31.37   
                            09            26458.92           19.87   
                            10            24726.42           24.68   
                            11            16754.15           59.15   
                            12            14711.59           98.73   

                                  HeatingEnergy(therms)  CoolingEnergy(kWh)  \
Scenario                   Month                                              
Clear/Clear                 01                   278.70                0.00   
                            02                   172.16                0.00   
                            03                   116.85                9.61   
                            04                    64.81              116.57   
                            05                    10.48              342.93   
...                                                 ...                 ...   
lowE(2)/lowE(4)-argon-VB75  08                     0.13              334.31   
                            09                    14.10              142.12   
                            10                    54.36                0.00   
                            11                   130.27                0.00   
                            12                   217.44                0.00   

                                  FanEnergy($)  HeatingEnergy($)  \
Scenario                   Month                                   
Clear/Clear                 01           16.63            231.32   
                            02           10.27            142.90   
                            03            7.11             96.99   
                            04            5.36             53.79   
                            05            5.03              8.70   
...                                        ...               ...   
lowE(2)/lowE(4)-argon-VB75  08            4.12              0.11   
                            09            2.61             11.70   
                            10            3.24             45.12   
                            11            7.77            108.12   
                            12           12.97            180.47   

                                  CoolingEnergy($)  MonthlyCost($)  \
Scenario                   Month                                     
Clear/Clear                 01                0.00          247.95   
                            02                0.00          153.17   
                            03                1.26          105.36   
                            04               15.32           74.47   
                            05               45.06           58.79   
...                                            ...             ...   
lowE(2)/lowE(4)-argon-VB75  08               43.93           48.16   
                            09               18.67           32.99   
                            10                0.00           48.36   
                            11                0.00          115.90   
                            12                0.00          193.45   

                                  DeltaFanFromBase  DeltaCoolFromBase  \
Scenario                   Month                                        
Clear/Clear                 01             -611.58           -2681.23   
                            02             -659.96           -2681.23   
       

In [50]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [51]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/msp_monthly_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [52]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [53]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

#MonthlyIlluminance.mean()

In [54]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/msp_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [55]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [56]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [57]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [58]:
# Convert Joules to kWh and therms in new dataframe
get_hourly["FanEnergy(kWh)"] = get_hourly["FanEnergy[J](Hourly)"] * .000000277777778
get_hourly["HeatingEnergy(therms)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_hourly["CoolingEnergy(kWh)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_hourly

In [59]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

------

## Get Hourly Cost Data
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [60]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [61]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [62]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanEnergy($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingEnergy($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingEnergy($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [63]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Establish baseline <br>
Export combine file

In [64]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner")

#allHourlyData

In [65]:
# Add energy cost totals

totalCosthourly = allHourlyData["FanEnergy($)"] + allHourlyData["HeatingEnergy($)"] + allHourlyData["CoolingEnergy($)"]
allHourlyData["HourlyCost($)"] = totalCosthourly

allHourlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($)
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.119879,0.264020,0.0,0.015752,0.219137,0.0,0.234889
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.134156,0.295466,0.0,0.017628,0.245237,0.0,0.262865
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.137411,0.302632,0.0,0.018056,0.251185,0.0,0.269241
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.137031,0.301797,0.0,0.018006,0.250491,0.0,0.268497
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.139573,0.307395,0.0,0.018340,0.255138,0.0,0.273478
...,...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.080704,0.177742,0.0,0.010604,0.147526,0.0,0.158130
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.080372,0.177011,0.0,0.010561,0.146919,0.0,0.157480
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.083224,0.183292,0.0,0.010936,0.152132,0.0,0.163068
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.086980,0.191564,0.0,0.011429,0.158998,0.0,0.170427



### Establish Hourly Energy and Cost Baseline Data


In [66]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_hourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_hourly

In [67]:
hourlyEnergy_fan = baseline_hourly["FanEnergy(kWh)"].mean()
hourlyEnergy_fan

0.08426108447041908

In [68]:
hourlyEnergy_cool = baseline_hourly["CoolingEnergy(kWh)"].mean()
hourlyEnergy_cool

0.3060763500734302

In [69]:
hourlyEnergy_heat = baseline_hourly["HeatingEnergy(therms)"].mean()
hourlyEnergy_heat

0.12156471368365519

In [70]:
hourlyCost_fan = baseline_hourly["FanEnergy($)"].mean()
hourlyCost_fan

0.011071906499413016

In [71]:
hourlyCost_cool = baseline_hourly["CoolingEnergy($)"].mean()
hourlyCost_cool

0.04021843239964872

In [72]:
hourlyCost_heat = baseline_hourly["HeatingEnergy($)"].mean()
hourlyCost_heat

0.10089871235743367

In [73]:
hourlyCost = (hourlyCost_fan + hourlyCost_cool + hourlyCost_heat)
hourlyCost

0.1521890512564954

## Export Hourly Data

Add baselines to dataframe<br>
Export to file

In [74]:
# Add columns for delta from base
allHourlyData["DeltaFanFromBase"] = allHourlyData["FanEnergy(kWh)"] - hourlyEnergy_fan
allHourlyData["DeltaCoolFromBase"] = allHourlyData["CoolingEnergy(kWh)"] - hourlyEnergy_cool
allHourlyData["DeltaHeatFromBase"] = allHourlyData["HeatingEnergy(therms)"] - hourlyEnergy_heat
allHourlyData["DeltaCostFromBase"] = allHourlyData["HourlyCost($)"] - hourlyCost

allHourlyData.round(2)

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($),DeltaFanFromBase,DeltaCoolFromBase,DeltaHeatFromBase,DeltaCostFromBase
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.12,0.26,0.0,0.02,0.22,0.0,0.23,0.04,-0.31,0.14,0.08
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.13,0.30,0.0,0.02,0.25,0.0,0.26,0.05,-0.31,0.17,0.11
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.14,0.30,0.0,0.02,0.25,0.0,0.27,0.05,-0.31,0.18,0.12
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.14,0.30,0.0,0.02,0.25,0.0,0.27,0.05,-0.31,0.18,0.12
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.14,0.31,0.0,0.02,0.26,0.0,0.27,0.06,-0.31,0.19,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716955,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1716956,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1716957,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1716958,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.09,0.19,0.0,0.01,0.16,0.0,0.17,0.00,-0.31,0.07,0.02


In [75]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/msp_daily.csv", header=True, index=False)